Codes needed for reading the files and preparing them for language model generation

In [ ]:
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def extact_sentences(pattern, file_path):
  # pattern:  is the regular expression you have defined to extract sentences
  # file_path:path to the trascript file 

  # key: Jury value: sentence said
  dictionary_lines = {}
  with open(file_path, 'r', encoding="unicode_escape") as file:
    line = file.readline()
    while(line):
      # this is where you will check each line
      ###
      ###
      
      line = file.readline()

  # Printing the number of sentences said by each jury (just for checking)
  for jury in dictionary_lines:
    print(jury + ":\t" + str(len(dictionary_lines[jury])))

def pre_process_inputs(dictionary_lines):
  # This method takes in the dictionary from previous method
  # It will return two dictionaries: one having all the tokens along
  # with their frequencies (list to counter?)
  # The other dictionary will have jury as the key (e.g., NO.2)
  # and the values are another dictionary of {token: frequency} for each jury
  
  lst_all_words = []
  dictionary_juror_words = {}
  for jury in dictionary_lines:
    # Here update all the words list and the dictionary for each jury
    lst_sentences_by_jury = dictionary_lines[jury]
    words_jorur = []
    for sentence in lst_sentences:
      ### Your code goes here (remove pass)
      pass
    dictionary_juror_words[jury] = dict(Counter(words_jorur)) 
  
  vocabulary = dict(Counter(lst_all_words))
  return vocabulary, dictionary_juror_words

Code to generate languge model

In [ ]:
from collections import Counter

def unigram_lm(dic_vocabulary, dic_juror_words):
  # This method will generate the unigram language model for each juror
  # The return value is a dict with juror as key and value as his language model
  # the inputs are the two dictionaries from pre_process_inputs method
  dic_unigram_lm = {}

  for jury in dic_juror_words:
    dic_words_frequency = dic_juror_words[jury]
    language_model = {key: 0 for key in dic_vocabulary}
    # Here you will calculate the probabilities with MLE and add-one smoothing
    for key in language_model:
      ### Remove pass
      pass
  return dic_unigram_lm

Code to generate T-SNE plot

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import numpy as np


def tsne_generator(dic_unigram_lm):
  tsne = TSNE(n_components=2)
  lst_freq = []
  lst_names = []
  for juror in dic_unigram_lm:
    lst_names.append(juror)
    
    # frequencies is a dictionary of token: frequencies
    # sort this by key and then convert values to the list and append it 
    # to the lst_freq
    frequencies = dic_unigram_lm[juror]
    ##### 
    # Your code goes here and then remove freq=[0] and instead of [0]
    # save your results in freq
    freq = [0]
    lst_freq.append(freq)
  
  # Convert the list to a NumPy array
  np_array = np.array(lst_freq)
  data = np_array

  # getting vectors with tsne
  vectors = tsne.fit_transform(data)

  # Your code for plot goes here
  # fig, ax = plt.subplots()
  # Some codes here to define what to be shown
  ####
  # making sure the legend is shown (uncomment)
  #ax.legend(bbox_to_anchor=(1.1, 1.05))
  #ax.grid(True)
  #plt.show()